### The Broadcasting
The term broadcasting describes how numpy treats arrays with different shapes during arithmetic operations. Subject to certain constraints, the smaller array is “broadcast” across the larger array so that they have compatible shapes. Broadcasting provides a means of vectorizing array operations so that looping occurs in C instead of Python. It does this without making needless copies of data and usually leads to efficient algorithm implementations. There are, however, cases where broadcasting is a bad idea because it leads to inefficient use of memory that slows computation.

NumPy operations are usually done on pairs of arrays on an element-by-element basis. In the simplest case, the two arrays must have exactly the same shape, as in the following example:

In [1]:
import numpy as np

a = np.array([1.0,2.0,3.0])
b = np.array([5.0,6.0,7.0])

a*b

array([ 5., 12., 21.])

NumPy’s broadcasting rule relaxes this constraint when the arrays’ shapes meet certain constraints. The simplest broadcasting example occurs when an array and a scalar value are combined in an operation:

In [2]:
b = np.array([5.0,6.0,7.0])
c = 5.0

c*b

array([25., 30., 35.])

The result is equivalent to the previous example where b was an array. We can think of the scalar b being stretched during the arithmetic operation into an array with the same shape as a. The new elements in b are simply copies of the original scalar. The stretching analogy is only conceptual. NumPy is smart enough to use the original scalar value without actually making copies so that broadcasting operations are as memory and computationally efficient as possible.

The code in the second example is more efficient than that in the first because broadcasting moves less memory around during the multiplication (b is a scalar rather than an array).

### General Broadcasting Rules
When operating on two arrays, NumPy compares their shapes element-wise. It starts with the trailing dimensions and works its way forward. Two dimensions are compatible when

they are equal, or

one of them is 1

If these conditions are not met, a ValueError: operands could not be broadcast together exception is thrown, indicating that the arrays have incompatible shapes. The size of the resulting array is the size that is not 1 along each axis of the inputs.

Arrays do not need to have the same number of dimensions. For example, if you have a 256x256x3 array of RGB values, and you want to scale each color in the image by a different value, you can multiply the image by a one-dimensional array with 3 values. Lining up the sizes of the trailing axes of these arrays according to the broadcast rules, shows that they are compatible:

In [3]:
x = np.arange(4)
xx = x.reshape(4,1)
y = np.ones(5)
z = np.ones((3,4))

In [4]:
x.shape

(4,)

In [5]:
y.shape

(5,)

In [6]:
x+y

ValueError: operands could not be broadcast together with shapes (4,) (5,) 

In [ ]:
xx.shape

In [ ]:
y.shape

In [ ]:
xx+y

In [ ]:
(xx+y).shape

In [ ]:
x.shape

In [ ]:
z.shape

In [ ]:
(x+z).shape

In [ ]:
x+z

Broadcasting provides a convenient way of taking the outer product (or any other outer operation) of two arrays. The following example shows an outer addition operation of two 1-d arrays:

In [ ]:
a = np.array([0.0, 10.0, 20.0, 30.0])
b = np.array([1.0, 2.0, 3.0])
a[:, np.newaxis] + b

## Structured arrays
### Introduction
Structured arrays are ndarrays whose datatype is a composition of simpler datatypes organized as a sequence of named fields. For example,

In [ ]:
import numpy as np

In [ ]:
x = np.array([("Neil",23,40.0), ("Shruti", 18, 37)],\
             dtype=[("Name", "U10"),("Age","i4"),("Weight","f4")])
x

Here x is a one-dimensional array of length two whose datatype is a structure with three fields: 1. A string of length 10 or less named ‘name’, 2. a 32-bit integer named ‘age’, and 3. a 32-bit float named ‘weight’.

If you index x at position 1 you get a structure:

In [ ]:
x[1]

You can access and modify individual fields of a structured array by indexing with the field name:

In [ ]:
x['Age']

In [ ]:
x['Age']=22
x

Structured datatypes are designed to be able to mimic ‘structs’ in the C language, and share a similar memory layout. They are meant for interfacing with C code and for low-level manipulation of structured buffers, for example for interpreting binary blobs. For these purposes they support specialized features such as subarrays, nested datatypes, and unions, and allow control over the memory layout of the structure.

Users looking to manipulate tabular data, such as stored in csv files, may find other pydata projects more suitable, such as xarray, pandas, or DataArray. These provide a high-level interface for tabular data analysis and are better optimized for that use. For instance, the C-struct-like memory layout of structured arrays in numpy can lead to poor cache behavior in comparison.

## Structured Datatypes
A structured datatype can be thought of as a sequence of bytes of a certain length (the structure’s itemsize) which is interpreted as a collection of fields. Each field has a name, a datatype, and a byte offset within the structure. The datatype of a field may be any numpy datatype including other structured datatypes, and it may also be a subarray data type which behaves like an ndarray of a specified shape. The offsets of the fields are arbitrary, and fields may even overlap. These offsets are usually determined automatically by numpy, but can also be specified.

### Structured Datatype Creation
Structured datatypes may be created using the function numpy.dtype. There are 4 alternative forms of specification which vary in flexibility and conciseness. These are further documented in the Data Type Objects reference page, and in summary they are:

1. A list of tuples, one tuple per field:   

 Each tuple has the form (fieldname, datatype, shape) where shape is optional. fieldname is a string (or tuple if titles are used, see Field Titles below), datatype may be any object convertible to a datatype, and shape is a tuple of integers specifying subarray shape.

In [ ]:
np.dtype([("Name", "U10"),("Age", "i4"),\
          ("Weight",np.float64),("array",np.float32,(2,2))])


If fieldname is the empty string '', the field will be given a default name of the form f#, where # is the integer index of the field, counting from 0 from the left:

In [ ]:
np.dtype([("Name",np.float64),("", np.int64), ("Age", np.int64)])

The byte offsets of the fields within the structure and the total structure itemsize are determined automatically.

2. A string of comma-separated dtype specifications:

 In this shorthand notation any of the string dtype specifications may be used in a string and separated by commas. The itemsize and byte offsets of the fields are determined automatically, and the field names are given the default names f0, f1, etc.

In [ ]:
np.dtype("i4,f4,S3")

In [ ]:
np.dtype("i4, 3f4, (2,2)f4")

3. A dictionary of field parameter arrays:

 This is the most flexible form of specification since it allows control over the byte-offsets of the fields and the itemsize of the structure.

 The dictionary has two required keys, ‘names’ and ‘formats’, and four optional keys, ‘offsets’, ‘itemsize’, ‘aligned’ and ‘titles’. The values for ‘names’ and ‘formats’ should respectively be a list of field names and a list of dtype specifications, of the same length. The optional ‘offsets’ value should be a list of integer byte-offsets, one for each field within the structure. If ‘offsets’ is not given the offsets are determined automatically. The optional ‘itemsize’ value should be an integer describing the total size in bytes of the dtype, which must be large enough to contain all the fields.

In [ ]:
np.dtype({"names":["Col1","Col2"],"formats":["int64","float32"]})


In [ ]:
np.dtype({"names":["col1","col2"],\
         "formats":["int64", "float32"],\
         "offsets":[0,4],\
         "itemsize":12})

Offsets may be chosen such that the fields overlap, though this will mean that assigning to one field may clobber any overlapping field’s data. As an exception, fields of numpy.object type cannot overlap with other fields, because of the risk of clobbering the internal object pointer and then dereferencing it.

The optional ‘aligned’ value can be set to True to make the automatic offset computation use aligned offsets (see Automatic Byte Offsets and Alignment), as if the ‘align’ keyword argument of numpy.dtype had been set to True.

The optional ‘titles’ value should be a list of titles of the same length as ‘names’, see Field Titles below.

4. A dictionary of field names

 The use of this form of specification is discouraged, but documented here because older numpy code may use it. The keys of the dictionary are the field names and the values are tuples specifying type and offset:

In [ ]:
np.dtype({"col1":("int8",0), "col2":("float64", 6)})

This form is discouraged because Python dictionaries do not preserve order in Python versions before Python 3.6, and the order of the fields in a structured dtype has meaning. Field Titles may be specified by using a 3-tuple, see below.

### Manipulating and Displaying Structured Datatypes
The list of field names of a structured datatype can be found in the names attribute of the dtype object:

In [ ]:
d = np.dtype([("x1", "int64"),("x2", np.float128)])
d

In [ ]:
d.names

In [ ]:
d.fields

The field names may be modified by assigning to the names attribute using a sequence of strings of the same length.

The dtype object also has a dictionary-like attribute, fields, whose keys are the field names (and Field Titles, see below) and whose values are tuples containing the dtype and byte offset of each field.

Both the names and fields attributes will equal None for unstructured arrays. The recommended way to test if a dtype is structured is with if dt.names is not None rather than if dt.names, to account for dtypes with 0 fields.

The string representation of a structured datatype is shown in the “list of tuples” form if possible, otherwise numpy falls back to using the more general dictionary form.

### Automatic Byte Offsets and Alignment
Numpy uses one of two methods to automatically determine the field byte offsets and the overall itemsize of a structured datatype, depending on whether align=True was specified as a keyword argument to numpy.dtype.

By default (align=False), numpy will pack the fields together such that each field starts at the byte offset the previous field ended, and the fields are contiguous in memory.

In [ ]:
def print_offsets(d):
    print("The offsets",[d.fields[name][1] for name in d.names])
    print(str(d.itemsize))
    
print_offsets(np.dtype("u1,u2,i4,f4"))
    

If align=True is set, numpy will pad the structure in the same way many C compilers would pad a C-struct. Aligned structures can give a performance improvement in some cases, at the cost of increased datatype size. Padding bytes are inserted between fields such that each field’s byte offset will be a multiple of that field’s alignment, which is usually equal to the field’s size in bytes for simple datatypes, see PyArray_Descr.alignment. The structure will also have trailing padding added so that its itemsize is a multiple of the largest field’s alignment.

In [ ]:
print_offsets(np.dtype("u1,u2,i4,f4,u1", align =True))

Note that although almost all modern C compilers pad in this way by default, padding in C structs is C-implementation-dependent so this memory layout is not guaranteed to exactly match that of a corresponding struct in a C program. Some work may be needed, either on the numpy side or the C side, to obtain exact correspondence.

If offsets were specified using the optional offsets key in the dictionary-based dtype specification, setting align=True will check that each field’s offset is a multiple of its size and that the itemsize is a multiple of the largest field size, and raise an exception if not.

If the offsets of the fields and itemsize of a structured array satisfy the alignment conditions, the array will have the ALIGNED flag set.

A convenience function numpy.lib.recfunctions.repack_fields converts an aligned dtype or array to a packed one and vice versa. It takes either a dtype or structured ndarray as an argument, and returns a copy with fields re-packed, with or without padding bytes.

### Field Titles
In addition to field names, fields may also have an associated title, an alternate name, which is sometimes used as an additional description or alias for the field. The title may be used to index an array, just like a field name.

To add titles when using the list-of-tuples form of dtype specification, the field name may be specified as a tuple of two strings instead of a single string, which will be the field’s title and field name respectively. For example:

In [ ]:
np.array([(("name","fname"),"f4")])

When using the first form of dictionary-based specification, the titles may be supplied as an extra 'titles' key as described above. When using the second (discouraged) dictionary-based specification, the title can be supplied by providing a 3-element tuple (datatype, offset, title) instead of the usual 2-element tuple:

In [ ]:
np.dtype({'name': ('i4', 0, 'my title')})

The dtype.fields dictionary will contain titles as keys, if any titles are used. This means effectively that a field with a title will be represented twice in the fields dictionary. The tuple values for these fields will also have a third element, the field title. Because of this, and because the names attribute preserves the field order while the fields attribute may not, it is recommended to iterate through the fields of a dtype using the names attribute of the dtype, which will not list titles, as in:

In [ ]:
for name in d.names:
    print(d.fields[name][:2])

### Union types
Structured datatypes are implemented in numpy to have base type numpy.void by default, but it is possible to interpret other numpy types as structured types using the (base_dtype, dtype) form of dtype specification described in Data Type Objects. Here, base_dtype is the desired underlying dtype, and fields and flags will be copied from dtype. This dtype is similar to a ‘union’ in C.

### Indexing and Assignment to Structured arrays
### Assigning data to a Structured Array
There are a number of ways to assign values to a structured array: Using python tuples, using scalar values, or using other structured arrays.

### Assignment from Python Native Types (Tuples)
The simplest way to assign values to a structured array is using python tuples. Each assigned value should be a tuple of length equal to the number of fields in the array, and not a list or array as these will trigger numpy’s broadcasting rules. The tuple’s elements are assigned to the successive fields of the array, from left to right:

In [ ]:
import numpy as np

In [ ]:
a = np.array([(1,2,3),(4,5,6)], dtype=("i4,f4,f8"))
a

In [ ]:
a[1]

In [ ]:
a[1] = (9,8,0)
a

### Assignment from Scalars
A scalar assigned to a structured element will be assigned to all fields. This happens when a scalar is assigned to a structured array, or when an unstructured array is assigned to a structured array:

In [ ]:
b = np.zeros(2, dtype=("i4,i8,?,S1"))
b

In [ ]:
b[:] = 3
b

In [ ]:
b[:] = np.arange(2)
b

Structured arrays can also be assigned to unstructured arrays, but only if the structured datatype has just a single field:

In [ ]:
twofield = np.zeros(2, dtype=[("one","i8"),("two","i4")])
twofield

In [ ]:
onefield = np.zeros(2, dtype=[("one","i4")])
onefield

In [ ]:
nostruct = np.zeros(2, dtype="i4")
nostruct

In [ ]:
nostruct[:] = twofield

### Assignment from other Structured Arrays
Assignment between two structured arrays occurs as if the source elements had been converted to tuples and then assigned to the destination elements. That is, the first field of the source array is assigned to the first field of the destination array, and the second field likewise, and so on, regardless of field names. Structured arrays with a different number of fields cannot be assigned to each other. Bytes of the destination structure which are not included in any of the fields are unaffected.

In [ ]:
a = np.zeros(3, dtype=[("A", "i4"),("B", "f4"),("C","S3")])
a

In [ ]:
b = np.ones(3, dtype=[("X","f8"),("Y","S3"),("Z", "O")])
b

In [ ]:
b[:] = a
b

### Assignment involving subarrays
When assigning to fields which are subarrays, the assigned value will first be broadcast to the shape of the subarray.

### Indexing Structured Arrays
### Accessing Individual Fields
Individual fields of a structured array may be accessed and modified by indexing the array with the field name.

In [ ]:
x = np.array([(1,2),(3,4)], dtype=[("A", "i8"),("B","f8")])
x

In [ ]:
x["A"]

In [ ]:
x["A"] = 10
x

The resulting array is a view into the original array. It shares the same memory locations and writing to the view will modify the original array.

In [ ]:
y =x["B"]
y

In [ ]:
y[:] = 11
y

In [ ]:
x

This view has the same dtype and itemsize as the indexed field, so it is typically a non-structured array, except in the case of nested structures.

In [ ]:
y.shape, y.strides, y.dtype

If the accessed field is a subarray, the dimensions of the subarray are appended to the shape of the result:

In [ ]:
x1 = np.zeros((2,2), dtype=[("A",np.int64),("B",np.float64,(3,3))])
x1

In [ ]:
x1["A"].shape

In [ ]:
x1["B"].shape

In [ ]:
x1["A"]

In [ ]:
x1["B"]

### Accessing Multiple Fields¶
One can index and assign to a structured array with a multi-field index, where the index is a list of field names.

The result of indexing with a multi-field index is a view into the original array, as follows:

In [ ]:
a = np.zeros(3, dtype=[("A", "i4"),("B", "f8"),("C","i8")])
a

In [ ]:
a[["A","C"]]

Assignment to the view modifies the original array. The view’s fields will be in the order they were indexed. Note that unlike for single-field indexing, the dtype of the view has the same itemsize as the original array, and has fields at the same offsets as in the original array, and unindexed fields are merely missing.

Assignment to an array with a multi-field index modifies the original array:



In [ ]:
a[["A","C"]] = (90,90)
a

This obeys the structured array assignment rules described above. For example, this means that one can swap the values of two fields using appropriate multi-field indexes:

In [ ]:
a[["A","B"]] = a[["A","C"]]
a

Indexing with an Integer to get a Structured Scalar
Indexing a single element of a structured array (with an integer index) returns a structured scalar:

In [ ]:
x = np.array([1,2.0,3.0], dtype=("i4, f8, f4"))
x

In [ ]:
x[0]

In [ ]:
type(x[0])

Unlike other numpy scalars, structured scalars are mutable and act like views into the original array, such that modifying the scalar will modify the original array. Structured scalars also support access and assignment by field name:

In [ ]:
x = np.array([(1,2),(3,4)], dtype=[("A", np.int64), ("B", np.float64)])
x

In [ ]:
y = x[0]
y

In [ ]:
y['B'] = 100
y

In [ ]:
x

Similarly to tuples, structured scalars can also be indexed with an integer:

In [ ]:
scalar = np.array([(1,2,3)],dtype=("i, f, f"))[0]
scalar

In [ ]:
scalar[1]

In [ ]:
scalar[0] =100
scalar

Thus, tuples might be thought of as the native Python equivalent to numpy’s structured types, much like native python integers are the equivalent to numpy’s integer types. Structured scalars may be converted to a tuple by calling ndarray.item:

In [ ]:
scalar.item(), type(scalar.item())

### Viewing Structured Arrays Containing Objects
In order to prevent clobbering object pointers in fields of numpy.object type, numpy currently does not allow views of structured arrays containing objects.

### Structure Comparison
If the dtypes of two void structured arrays are equal, testing the equality of the arrays will result in a boolean array with the dimensions of the original arrays, with elements set to True where all fields of the corresponding structures are equal. Structured dtypes are equal if the field names, dtypes and titles are the same, ignoring endianness, and the fields are in the same order:

In [ ]:
a = np.zeros(2, dtype=[('a', 'i4'), ('b', 'i4')])
b = np.ones(2, dtype=[('a', 'i4'), ('b', 'i4')])
a == b

Currently, if the dtypes of two void structured arrays are not equivalent the comparison fails, returning the scalar value False. This behavior is deprecated as of numpy 1.10 and will raise an error or perform elementwise comparison in the future.

The < and > operators always return False when comparing void structured arrays, and arithmetic and bitwise operations are not supported.

### Record Arrays
As an optional convenience numpy provides an ndarray subclass, numpy.recarray, and associated helper functions in the numpy.rec submodule, that allows access to fields of structured arrays by attribute instead of only by index. Record arrays also use a special datatype, numpy.record, that allows field access by attribute on the structured scalars obtained from the array.

The simplest way to create a record array is with numpy.rec.array:

In [7]:
recordarray = np.rec.array([(1,2,"Hello"),(2,3,"World")], dtype=[('foo', 'i4'),('bar', 'f4'), ('baz', 'S10')])
recordarray

rec.array([(1, 2., b'Hello'), (2, 3., b'World')],
          dtype=[('foo', '<i4'), ('bar', '<f4'), ('baz', 'S10')])

In [8]:
recordarray[0]

(1, 2., b'Hello')

In [9]:
recordarray[1:2].foo

array([2], dtype=int32)

In [10]:
recordarray[1].baz

b'World'

In [13]:
recordarray.baz[0]

b'Hello'

In [14]:
recordarray.foo[0]

1

numpy.rec.array can convert a wide variety of arguments into record arrays, including structured arrays:

In [15]:
arr = np.array([(1,2,"Hello"),(3,4,"World")], dtype=[("foo", "i4"),("bar", "f8"),("baz", "S10")])
arr

array([(1, 2., b'Hello'), (3, 4., b'World')],
      dtype=[('foo', '<i4'), ('bar', '<f8'), ('baz', 'S10')])

In [16]:
recordarr = np.rec.array(arr)
recordarr

rec.array([(1, 2., b'Hello'), (3, 4., b'World')],
          dtype=[('foo', '<i4'), ('bar', '<f8'), ('baz', 'S10')])

The numpy.rec module provides a number of other convenience functions for creating record arrays, see record array creation routines.
https://numpy.org/doc/1.19/reference/routines.array-creation.html#routines-array-creation-rec

A record array representation of a structured array can be obtained using the appropriate view:

In [17]:
arr = np.array([(1, 2., 'Hello'), (2, 3., "World")],\
               dtype=[('foo', 'i4'),('bar', 'f4'), ('baz', 'a10')])
recordarr = arr.view(dtype=np.dtype((np.record, arr.dtype)),\
                     type=np.recarray)

In [18]:
recordarr

rec.array([(1, 2., b'Hello'), (2, 3., b'World')],
          dtype=[('foo', '<i4'), ('bar', '<f4'), ('baz', 'S10')])

For convenience, viewing an ndarray as type np.recarray will automatically convert to np.record datatype, so the dtype can be left out of the view:

In [19]:
recordarr = arr.view(np.recarray)
recordarr.dtype

dtype((numpy.record, [('foo', '<i4'), ('bar', '<f4'), ('baz', 'S10')]))

To get back to a plain ndarray both the dtype and type must be reset. The following view does so, taking into account the unusual case that the recordarr was not a structured type:

In [22]:
arr2 = recordarr.view(recordarr.dtype.fields or recordarr.dtype, np.ndarray)

In [23]:
arr2

array([(1, 2., b'Hello'), (2, 3., b'World')],
      dtype=[('foo', '<i4'), ('bar', '<f4'), ('baz', 'S10')])

In [24]:
arr2 = recordarr.view(recordarr.dtype.fields, np.ndarray) # both work properly

Record array fields accessed by index or by attribute are returned as a record array if the field has a structured type but as a plain ndarray otherwise.

In [25]:
recordarr = np.rec.array([('Hello', (1, 2)), ("World", (3, 4))],\
                         dtype=[('foo', 'S6'),('bar', [('A', int), ('B', int)])])

In [26]:
type(recordarr.foo)

numpy.ndarray

In [27]:
type(recordarr.bar)

numpy.recarray

Note that if a field has the same name as an ndarray attribute, the ndarray attribute takes precedence. Such fields will be inaccessible by attribute but will still be accessible by index.

### Recarray Helper Functions
Collection of utilities to manipulate structured arrays.

Most of these functions were initially implemented by John Hunter for matplotlib. They have been rewritten and extended for convenience.

numpy.lib.recfunctions.append_fields(base, names, data, dtypes=None, fill_value=-1, usemask=True, asrecarray=False)

Add new fields to an existing array.

The names of the fields are given with the names arguments, the corresponding values with the data arguments. If a single field is appended, names, data and dtypes do not have to be lists but just values.

numpy.lib.recfunctions.apply_along_fields(func, arr)

Apply function ‘func’ as a reduction across fields of a structured array.

This is similar to apply_along_axis, but treats the fields of a structured array as an extra axis. The fields are all first cast to a common type following the type-promotion rules from numpy.result_type applied to the field’s dtypes.

In [29]:
from numpy.lib import recfunctions as rfc
import numpy as np

In [30]:
b = np.array([(1,2,3),(4,5,6),(7,8,9),(10,11,12),(13,14,15)],\
             dtype=[("A", "i4"),("B","f8"),("C","i8")])
b

array([( 1,  2.,  3), ( 4,  5.,  6), ( 7,  8.,  9), (10, 11., 12),
       (13, 14., 15)], dtype=[('A', '<i4'), ('B', '<f8'), ('C', '<i8')])

In [31]:
rfc.apply_along_fields(np.mean, b)

array([ 2.,  5.,  8., 11., 14.])

In [32]:
rfc.apply_along_fields(np.mean, b[["A","B"]])

array([ 1.5,  4.5,  7.5, 10.5, 13.5])

numpy.lib.recfunctions.assign_fields_by_name(dst, src, zero_unassigned=True)

Assigns values from one structured array to another by field name.

numpy.lib.recfunctions.drop_fields(base, drop_names, usemask=True, asrecarray=False)

Return a new array with fields in drop_names dropped.

Nested fields are supported.

In [34]:
from numpy.lib import recfunctions as rfc

In [35]:
a = np.array([(1,(2,3)),(4,(5,6)),(7,(8,9))], \
             dtype=[("A", "i8"),("B",[("ba", np.float32),("bc", np.int64)])])
a

array([(1, (2., 3)), (4, (5., 6)), (7, (8., 9))],
      dtype=[('A', '<i8'), ('B', [('ba', '<f4'), ('bc', '<i8')])])

In [36]:
rfc.drop_fields(a,'a')

array([(1, (2., 3)), (4, (5., 6)), (7, (8., 9))],
      dtype=[('A', '<i8'), ('B', [('ba', '<f4'), ('bc', '<i8')])])

In [37]:
rfc.drop_fields(a,'ba')

array([(1, (3,)), (4, (6,)), (7, (9,))],
      dtype=[('A', '<i8'), ('B', [('bc', '<i8')])])

In [38]:
rfc.drop_fields(a,['ba','bc'])

array([(1,), (4,), (7,)], dtype=[('A', '<i8')])

numpy.lib.recfunctions.find_duplicates(a, key=None, ignoremask=True, return_index=False)

Find the duplicates in a structured array along a given key

In [40]:
from numpy.lib import recfunctions as rfn
ndtype = [('a', int)]
a = np.ma.array([1, 1, 1, 2, 2, 3, 3],\
                mask=[0, 0, 1, 0, 0, 0, 1]).view(ndtype)

rfn.find_duplicates(a, ignoremask=True, return_index=True)

(masked_array(data=[(1,), (1,), (2,), (2,)],
              mask=[(False,), (False,), (False,), (False,)],
        fill_value=(999999,),
             dtype=[('a', '<i8')]),
 array([0, 1, 3, 4]))

In [41]:
from numpy.lib import recfunctions as rfn
ndtype = np.dtype([('a', '<i4'), ('b', [('ba', '<f8'), ('bb', '<i4')])])
rfn.flatten_descr(ndtype)

(('a', dtype('int32')), ('ba', dtype('float64')), ('bb', dtype('int32')))

numpy.lib.recfunctions.get_fieldstructure(adtype, lastname=None, parents=None)

Returns a dictionary with fields indexing lists of their parent fields.

This function is used to simplify access to fields nested in other fields.

In [42]:
from numpy.lib import recfunctions as rfn
ndtype =  np.dtype([('A', int),
                    ('B', [('BA', int),
                           ('BB', [('BBA', int), ('BBB', int)])])])
rfn.get_fieldstructure(ndtype)

{'A': [],
 'B': [],
 'BA': ['B'],
 'BB': ['B'],
 'BBA': ['B', 'BB'],
 'BBB': ['B', 'BB']}

numpy.lib.recfunctions.join_by(key, r1, r2, jointype='inner', r1postfix='1', r2postfix='2', defaults=None, usemask=True, asrecarray=False)

Join arrays r1 and r2 on key key.

The key should be either a string or a sequence of string corresponding to the fields used to join the array. An exception is raised if the key field cannot be found in the two input arrays. Neither r1 nor r2 should have any duplicates along key: the presence of duplicates will make the output quite unreliable. Note that duplicates are not looked for by the algorithm.

Notes

The output is sorted along the key.

A temporary array is formed by dropping the fields not in the key for the two arrays and concatenating the result. This array is then sorted, and the common entries selected. The output is constructed by filling the fields with the selected entries. Matching is not preserved if there are some duplicates…

numpy.lib.recfunctions.merge_arrays(seqarrays, fill_value=-1, flatten=False, usemask=False, asrecarray=False)

Merge arrays field by field.

Notes

Without a mask, the missing value will be filled with something, depending on what its corresponding type:

-1 for integers

-1.0 for floating point numbers

'-' for characters

'-1' for strings

True for boolean values

XXX: I just obtained these values empirically

In [43]:
from numpy.lib import recfunctions as rfn
rfn.merge_arrays((np.array([1, 2]), np.array([10., 20., 30.])))

array([( 1, 10.), ( 2, 20.), (-1, 30.)],
      dtype=[('f0', '<i8'), ('f1', '<f8')])

In [46]:
rfn.merge_arrays((np.array([1, 2], dtype=np.int64),\
                  np.array([10., 20., 30.])), usemask=False),\
                  np.array([(1, 10.0), (2, 20.0), (-1, 30.0)],\
                        dtype=[('f0', '<i8'), ('f1', '<f8')])

(array([( 1, 10.), ( 2, 20.), (-1, 30.)],
       dtype=[('f0', '<i8'), ('f1', '<f8')]),
 array([( 1, 10.), ( 2, 20.), (-1, 30.)],
       dtype=[('f0', '<i8'), ('f1', '<f8')]))

In [47]:
rfn.merge_arrays((np.array([1, 2]).view([('a', np.int64)]),
              np.array([10., 20., 30.])),
             usemask=False, asrecarray=True)

rec.array([( 1, 10.), ( 2, 20.), (-1, 30.)],
          dtype=[('a', '<i8'), ('f1', '<f8')])

numpy.lib.recfunctions.rename_fields(base, namemapper)
Rename the fields from a flexible-datatype ndarray or recarray.

Nested fields are supported.

In [50]:
from numpy.lib import recfunctions as rfn
a = np.array([(1, (2, [3.0, 30.])), (4, (5, [6.0, 60.]))],
  dtype=[('a', int),('b', [('ba', float), ('bb', (float, 2))])])
rfn.rename_fields(a, {'a':'A', 'bb':'BB'})

array([(1, (2., [ 3., 30.])), (4, (5., [ 6., 60.]))],
      dtype=[('A', '<i8'), ('b', [('ba', '<f8'), ('BB', '<f8', (2,))])])

numpy.lib.recfunctions.stack_arrays(arrays, defaults=None, usemask=True, asrecarray=False, autoconvert=False)[source]
Superposes arrays fields by fields

In [51]:
from numpy.lib import recfunctions as rfn
x = np.array([1, 2,])
rfn.stack_arrays(x) is x

True

In [52]:
z = np.array([('A', 1), ('B', 2)], dtype=[('A', '|S3'), ('B', float)])
zz = np.array([('a', 10., 100.), ('b', 20., 200.), ('c', 30., 300.)],\
              dtype=[('A', '|S3'), ('B', np.double), ('C', np.double)])
test = rfn.stack_arrays((z,zz))
test

masked_array(data=[(b'A', 1.0, --), (b'B', 2.0, --), (b'a', 10.0, 100.0),
                   (b'b', 20.0, 200.0), (b'c', 30.0, 300.0)],
             mask=[(False, False,  True), (False, False,  True),
                   (False, False, False), (False, False, False),
                   (False, False, False)],
       fill_value=(b'N/A', 1.e+20, 1.e+20),
            dtype=[('A', 'S3'), ('B', '<f8'), ('C', '<f8')])

numpy.lib.recfunctions.structured_to_unstructured(arr, dtype=None, copy=False, casting='unsafe')
Converts and n-D structured array into an (n+1)-D unstructured array.

The new array will have a new last dimension equal in size to the number of field-elements of the input array. If not supplied, the output datatype is determined from the numpy type promotion rules applied to all the field datatypes.

Nested fields, as well as each element of any subarray fields, all count as a single field-elements.

In [53]:
from numpy.lib import recfunctions as rfn
a = np.zeros(4, dtype=[('a', 'i4'), ('b', 'f4,u2'), ('c', 'f4', 2)])
a

array([(0, (0., 0), [0., 0.]), (0, (0., 0), [0., 0.]),
       (0, (0., 0), [0., 0.]), (0, (0., 0), [0., 0.])],
      dtype=[('a', '<i4'), ('b', [('f0', '<f4'), ('f1', '<u2')]), ('c', '<f4', (2,))])

In [54]:
rfn.structured_to_unstructured(a)

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [55]:
b = np.array([(1, 2, 5), (4, 5, 7), (7, 8 ,11), (10, 11, 12)],
             dtype=[('x', 'i4'), ('y', 'f4'), ('z', 'f8')])
np.mean(rfn.structured_to_unstructured(b[['x', 'z']]), axis=-1)

array([ 3. ,  5.5,  9. , 11. ])

numpy.lib.recfunctions.unstructured_to_structured(arr, dtype=None, names=None, align=False, copy=False, casting='unsafe')
Converts and n-D unstructured array into an (n-1)-D structured array.

The last dimension of the input array is converted into a structure, with number of field-elements equal to the size of the last dimension of the input array. By default all output fields have the input array’s dtype, but an output structured dtype with an equal number of fields-elements can be supplied instead.

Nested fields, as well as each element of any subarray fields, all count towards the number of field-elements.

In [57]:
from numpy.lib import recfunctions as rfn
dt = np.dtype([('a', 'i4'), ('b', 'f4,u2'), ('c', 'f4', 2)])
a = np.arange(20).reshape((4,5))
a

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19]])

In [58]:
rfn.unstructured_to_structured(a, dt)

array([( 0, ( 1.,  2), [ 3.,  4.]), ( 5, ( 6.,  7), [ 8.,  9.]),
       (10, (11., 12), [13., 14.]), (15, (16., 17), [18., 19.])],
      dtype=[('a', '<i4'), ('b', [('f0', '<f4'), ('f1', '<u2')]), ('c', '<f4', (2,))])